In [1]:
import os

from gensim.models import CoherenceModel

os.environ['KERAS_BACKEND'] = "torch"

## Kickstarter Less Dataset (256k)


In [2]:
from train import AbaeModelConfiguration
from core.train import AbaeModelManager

config = AbaeModelConfiguration(
    corpus_file="../data/processed-dataset/full/64k.preprocessed.csv",
    model_name="abae.full.64k", aspect_size=16
)

manager = AbaeModelManager(config)

Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae.full.64k.embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae.full.64k.embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae.full.64k.embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae.full.64k.embeddings.keras', 'datetime': '2024-12-27T18:29:18.318914', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


In [3]:
inference_model = manager.prepare_evaluation_model()

D:\PycharmProjects\nlp-course-project\core\layer.py:126: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(WeightedAspectEmb, self).__init__(**kwargs)


## Top words for each aspect

In [4]:
from core.evaluation import normalize
from core.evaluation import get_aspect_top_k_words

word_emb = normalize(inference_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(inference_model.get_layer('weighted_aspect_emb').W)

print(f"Word embeddings shape: {word_emb.shape}")
inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab) for a in aspect_embeddings]

Word embeddings shape: torch.Size([9004, 128])

Given aspect most representative words are:
Word:  legendary (0.4351903200149536)
Word:  marvel (0.42146116495132446)
Word:  include (0.41859087347984314)
Word:  promos (0.40995022654533386)
Word:  universe (0.4064077138900757)
Word:  cards (0.3850122392177582)
Word:  expansions (0.3809184432029724)
Word:  unmatched (0.38064125180244446)
Word:  north (0.37879279255867004)
Word:  expansion (0.37542006373405457)
Word:  w/ (0.37347477674484253)
Word:  ✓ (0.36681437492370605)
Word:  <game_name> (0.36548852920532227)
Word:  promo (0.36144232749938965)
Word:  shrink (0.3540777862071991)
Word:  american (0.3519633412361145)
Word:  flight (0.3502519726753235)
Word:  pledge (0.34722787141799927)
Word:  hill (0.3468876779079437)
Word:  premium (0.346232533454895)
Word:  wars (0.34497418999671936)
Word:  exp (0.339688777923584)
Word:  cthulhu (0.33938536047935486)
Word:  wing (0.33663520216941833)
Word:  european (0.33345741033554077)

Given aspect 

![title](../resources/umass.png)


In [6]:
import dataset

vocab = manager.embedding_model.model.wv.key_to_index
ds = dataset.PositiveNegativeCommentGeneratorDataset(config.corpus_file, vocab, 15)

Loading dataset from file: ../data/processed-dataset/full/64k.preprocessed.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  206 . The limit is set to 256 tokens.
Padding sequences to length (256).


4.4. Word2vec Coherence Score
[https://www.baeldung.com/cs/topic-modeling-coherence-score#4-word2vec-coherence-score

In [9]:
from core.evaluation import coherence_per_aspect

aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
coherence_per_aspect(aspect_words, ds.text_ds, 20)

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

([-13.779072904076862,
  -15.277126910332,
  -15.260658495664517,
  -12.835287176738467,
  -10.315495960196573,
  -12.591657242036765,
  -11.790351217401147,
  -11.284792485753506,
  -11.191647439438862,
  -16.373899209806282,
  -11.957909535898537,
  -12.80126061494811,
  -17.199991888478674,
  -13.270242112937026,
  -12.375687470990492,
  -9.118723046246984],
 <gensim.models.coherencemodel.CoherenceModel at 0x238d70574d0>)

In [9]:
# todo: graph of the loss of coherence compared to word considered increase

[[('legendary',
   tensor(0.4352, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1099, device='cuda:0')),
  ('marvel',
   tensor(0.4215, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(986, device='cuda:0')),
  ('include',
   tensor(0.4186, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(152, device='cuda:0')),
  ('promos',
   tensor(0.4100, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1749, device='cuda:0')),
  ('universe',
   tensor(0.4064, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1247, device='cuda:0')),
  ('cards',
   tensor(0.3850, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1082, device='cuda:0')),
  ('expansions',
   tensor(0.3809, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(2420, device='cuda:0')),
  ('unmatched',
   tensor(0.3806, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1468, device='cuda:0')),
  ('north',
   tensor(0.3788, device='cuda:0', grad_fn=<SelectBackward0>),
   tensor(1991, device='cud